C:\Users\natha\Documents\School\FS22\CSE803\Project\py-sc\803-Final-Project\homography.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self.H = np.linalg.lstsq(x, y)[0]


In [19]:
    with open("./transformer.pickle", 'rb') as f:
        transformer = pickle.load(f)

In [20]:
transformer.x_scale

0.05

In [21]:
transformer.y_scale

0.05

In [13]:
x

array([[ 3.7407227 ,  1.7524414 , 11.988958  ],
       [ 2.7407227 ,  1.7526855 , 11.988958  ],
       [ 1.7409668 ,  1.7531738 , 11.988958  ],
       [ 0.74121094,  1.7539062 , 11.988958  ],
       [-0.25830078,  1.7546387 , 11.988958  ],
       [-1.2575684 ,  1.7558594 , 11.988958  ],
       [-1.2561035 ,  0.7573242 , 11.988958  ],
       [-1.2548828 , -0.24169922, 11.988958  ],
       [-1.2539062 , -1.2412109 , 11.988958  ],
       [-1.253418  , -2.2409668 , 11.988958  ],
       [-1.2529297 , -3.2407227 , 11.988958  ],
       [-1.2524414 , -4.2407227 , 11.988958  ]], dtype=float32)

In [6]:
t.homography_transform(x)

AttributeError: 'tuple' object has no attribute 'T'

In [25]:
y

array([[1.4355469 , 0.80859375],
       [1.3183594 , 0.80859375],
       [1.2011719 , 0.80859375],
       [1.0839844 , 0.80859375],
       [0.9667969 , 0.80859375],
       [0.8496094 , 0.80859375],
       [0.8496094 , 0.9160156 ],
       [0.8496094 , 1.0019531 ],
       [0.8496094 , 1.1132812 ],
       [0.8496094 , 1.2109375 ],
       [0.8496094 , 1.3203125 ],
       [0.8496094 , 1.4492188 ],
       [0.18945312, 0.10351562],
       [0.4609375 , 0.4296875 ],
       [0.41601562, 0.6269531 ],
       [0.8339844 , 0.08789062],
       [0.45507812, 0.83203125],
       [0.4453125 , 0.5175781 ],
       [0.5214844 , 0.83203125],
       [0.47070312, 0.43554688],
       [0.22070312, 0.2578125 ]], dtype=float32)

In [26]:
np.linalg.lstsq(x, y)

C:\Users\natha\AppData\Local\Temp\ipykernel_18752\1483668873.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.linalg.lstsq(x, y)


(array([[ 0.07231373, -0.03198319],
        [ 0.01374402, -0.05186034],
        [ 0.06692821,  0.06332152]], dtype=float32),
 array([1.3110583, 1.790185 ], dtype=float32),
 3,
 array([54.17134 , 18.383343, 11.046874], dtype=float32))

In [4]:
import os
import pathlib
import pandas as pd
import pickle
from absl import app
import numpy as np
import cv2
import pybboxes as pbx

from homography import Transformer
import HarvestAgent
from UnitId import sc_to_enumerate


def val_boxes(yolo_data, image_path, output_path):
    print(F"Validating boxes using {image_path}, output to {output_path}")
    img = cv2.imread(str(image_path))

    for box in yolo_data.tolist():
        if 0 < box[3] and 0 < box[4]:
            b = pbx.YoloBoundingBox(box[1], box[2], box[3], box[4], img.shape[0:2])
            b = pbx.convert_bbox(b, to_type="voc")
            cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 1)

    cv2.imwrite(str(output_path), img)


def convert_truth(truth_file, image_file, transformer):
    truth_path = os.fsdecode(truth_file)
    image_path = os.fsdecode(image_file)
    yolo_file_path = os.path.splitext(truth_path)[0] + '.txt'

    truth = pd.read_csv(truth_path)

    # Center observation to camera
    camera = truth.loc[truth['name'] == 'Camera']
    c_x, c_y = float(camera['pos.x']), float(camera['pos.y'])
    truth['pos.x'] -= c_x
    truth['pos.y'] -= c_y
    truth['pos.z'] -= 11.62

    # Transform to camera plane
    uvwh = transformer.homography_transform(truth[['pos.x', 'pos.y', 'pos.z', 'radius']].values)
    print(uvwh)
    truth['u'] = uvwh[:, 0]
    truth['v'] = uvwh[:, 1]
    truth['w'] = uvwh[:, 2]
    truth['h'] = uvwh[:, 3]

    # Drop camera row
    truth.drop(truth[truth['name'] == 'Camera'].index, inplace=True)

    # scId -> enumerated id
    truth['ind_type'] = truth['sc_type'].map(sc_to_enumerate)


    # Output the data
    yolo_data = truth[['ind_type', 'u', 'v', 'w', 'h']].values
    col_format = '%d', '%1.6f', '%1.6f', '%1.6f', '%1.6f'
    np.savetxt(yolo_file_path, yolo_data, fmt=col_format)

    # Visualize validation boxes
    print(truth_path)
    vis_output_path = os.path.splitext(truth_path)[0] + '.png'
    val_boxes(yolo_data, image_path, vis_output_path)


t = Transformer(H=np.array([[2.11552, 0.27027, -0.42066, 18.15], [0, -1.50956, -1.56417, 18.15], [0, 0.54053, -0.84132, 36.3]]), x_scale=0.05, y_scale=0.05, img_size=1024)
convert_truth('data/units_6100_gt.csv', 'data/tick_6100.png', t)

[[ 0.5         0.78838324  0.09291739  0.08501939]
 [ 0.16183196  0.37358682  0.21769888  0.16211465]
 [ 0.64486819  0.0443201   0.1628077   0.12255345]
 [ 0.00598491  0.28445264  0.21946854  0.15321592]
 [ 0.32440848  0.51769777  0.21861342  0.17697172]
 [ 0.84390385  0.11522838  0.15796831  0.12867804]
 [ 0.63055795 -0.2359458   0.14759637  0.0997488 ]
 [ 0.69185174  0.73128526  0.21144319  0.20019938]
 [ 0.7290464  -0.08747573  0.15192955  0.11154339]
 [ 0.85251674  0.75993734  0.12795663  0.12764753]
 [-0.42637363  0.21659022  0.13066724  0.08124112]
 [ 1.17889209  1.17976565  0.12970245  0.1592763 ]
 [-0.20183658  0.07694133  0.11890153  0.07392474]
 [-0.18592489  0.01188221  0.11583911  0.07063144]
 [ 0.88268569  0.22826651  0.10156162  0.08689001]
 [ 0.7304049   0.22826651  0.10594151  0.08689001]
 [ 1.05684778  0.07924402  0.09185697  0.07858676]
 [ 1.04344648  0.01012465  0.09003379  0.07487427]
 [ 1.26395385  1.25334121  0.12858326  0.16516958]
 [-0.27415257  0.18042882  0.12